In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, r2_score,  mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression


## Load data

In [2]:
for file_name in os.listdir("files"):
    if file_name.endswith('.csv'):
        file_path = os.path.join("files", file_name)

        df_name = os.path.splitext(file_name)[0]
        globals()[df_name] = pd.read_csv(file_path)


## Simplification of categories

In [3]:
# Merge sports data and extract sports categories
def extract_sports_category(dataset, sports_code):
    merged = pd.merge(dataset, sports_code, left_on="Sports", right_on="Code")
    merged["Sports_Category"] = merged["Categorie"]
    return merged[["PRIMARY_KEY", "Sports_Category"]]

learn_sports = extract_sports_category(learn_dataset_sport, code_Sports)
test_sports = extract_sports_category(test_dataset_sport, code_Sports)

# Merge departments into regions and extract relevant region columns
def merge_and_extract_region(df, merge_column, region_column_name):
    merged = pd.merge(df, departments, left_on=merge_column, right_on="DEP", how="left")
    merged[region_column_name] = merged["REG"]
    return merged.drop(["Nom du département", "REG", "DEP", merge_column], axis=1)

In [4]:
learn_dataset_job = merge_and_extract_region(learn_dataset_job, "JOB_DEP", "JOB_REG")    #put the column as int 
learn_dataset_retired_jobs = merge_and_extract_region(learn_dataset_retired_jobs, "JOB_DEP", "JOB_REG")
learn_dataset_retired_jobs = merge_and_extract_region(learn_dataset_retired_jobs, "FORMER_DEP", "FORMER_REG")

test_dataset_job = merge_and_extract_region(test_dataset_job, "JOB_DEP", "JOB_REG")
test_dataset_retired_jobs = merge_and_extract_region(test_dataset_retired_jobs, "JOB_DEP", "JOB_REG")
test_dataset_retired_jobs = merge_and_extract_region(test_dataset_retired_jobs, "FORMER_DEP", "FORMER_REG")

In [5]:
#Economic sector into fewer categories (and numeric instead of object/string)
def sector_mapping(nace_code):
    if nace_code == "AZ":  
        return "Agriculture, forestry and fishing)"
    elif "BZ" <= nace_code <= "EZ":
        return "Manufacturing, mining and quarrying and other industrial activities"
    elif nace_code == "FZ": 
        return "Construction"
    elif "GZ" <= nace_code <= "IZ":  
        return "Wholesale and retail trade, transportation and storage, accommodation and food service activities"
    elif "JA" <= nace_code <= "JC":
        return "Information and communication"
    elif nace_code == "KZ": 
        return "Financial and insurance activities"
    elif nace_code == "LZ": 
        return "Real estate activities"
    elif "MA" <= nace_code <= "NZ":
        return "Professional, scientific, technical, administrative and support service activities"
    elif "OZ" <= nace_code <= "QB":
        return "Public administration and defence, education, human health and social work activities"
    elif "RZ" <= nace_code <= "UZ":
        return "Other services activities"
    else:
        return "Unknown Sector"

In [6]:
code_Economic_sector["Nomenclature"] = code_Economic_sector["Code"].map(sector_mapping)
code_Economic_sector["Economic_sector_num"] = pd.factorize(code_Economic_sector["Nomenclature"])[0] + 1

#issue in the code IZ should be its own category "Hébergement et restauration"
#HZ, JZ à MC so LZ, KZ, MA, MB and MC, NZ, OZ, PZ,  QA, QB, RZ, SZ, TZ et UZ should be in the same category

In [7]:
code_HIGHEST_CREDENTIAL["HIGHEST_CREDENTIAL_num"] = pd.factorize(code_HIGHEST_CREDENTIAL["Code"])[0] + 1
code_act["act_num"] = pd.factorize(code_act["Code"])[0] + 1

## Merging

In [8]:
learn_data = learn_dataset

learn_data = pd.merge(learn_data, code_act, left_on="act", right_on="Code", how="left")
learn_data.drop(["Code", "Libellé"], axis=1, inplace=True)  #should also drop act ?
learn_data = pd.merge(learn_data, code_HIGHEST_CREDENTIAL, left_on="HIGHEST_CREDENTIAL", right_on="Code", how="left")
learn_data.drop(["Code", "Libellé", "HIGHEST_CREDENTIAL"], axis=1, inplace=True)

#for imputation fitting
learn_data = pd.merge(learn_data, city_pop, on="INSEE_CODE", how="left")
learn_data = pd.merge(learn_data, city_loc, on="INSEE_CODE", how="left")

test_data = test_dataset

test_data = pd.merge(test_data, code_act, left_on="act", right_on="Code", how="left")
test_data.drop(["Code", "Libellé"], axis=1, inplace=True)
test_data = pd.merge(test_data, code_HIGHEST_CREDENTIAL, left_on="HIGHEST_CREDENTIAL", right_on="Code", how="left")
test_data.drop(["Code", "Libellé", "HIGHEST_CREDENTIAL"], axis=1, inplace=True)

test_data = pd.merge(test_data, city_pop, on="INSEE_CODE", how="left")
test_data = pd.merge(test_data, city_loc, on="INSEE_CODE", how="left")

In [9]:
learn_dfs = [learn_dataset_emp_contract, learn_dataset_job, learn_dataset_retired_former, 
             learn_dataset_retired_jobs, learn_dataset_retired_pension, learn_sports]

test_dfs = [test_dataset_emp_contract, test_dataset_job, test_dataset_retired_former, 
            test_dataset_retired_jobs, test_dataset_retired_pension, test_sports]

for df in learn_dfs:
    learn_data = pd.merge(learn_data, df, on="PRIMARY_KEY", how="outer")

for df in test_dfs:
    test_data = pd.merge(test_data, df, on="PRIMARY_KEY", how="outer")

In [10]:
# Function to combine columns ending with `_x` and `_y` into a single base column
def combine_duplicate_columns(dataframe):
    for column in dataframe.columns:
        if column.endswith('_x'):
            base_column = column[:-2]  # Remove `_x` suffix
            y_column = base_column + '_y'
            if y_column in dataframe.columns:
                # Combine the `_x` and `_y` columns
                dataframe[base_column] = dataframe[column].fillna(dataframe[y_column])
                # Drop the original `_x` and `_y` columns
                dataframe.drop(columns=[column, y_column], inplace=True)
    return dataframe

# Apply the function to both datasets
learn_data = combine_duplicate_columns(learn_data)
test_data = combine_duplicate_columns(test_data)


## Formatting 

In [11]:
def household_num(value):
    parts = value.split('|')  # Split the value by '|'
    if parts[1] in {'1', '2', '3'}:  # For M|1|-- to M|3|--
        return int(parts[1])
    elif parts[1] == '4':  # For M|4|1 to M|4|4
        return 4 + (int(parts[2]) - 1)  # 4 + (1-1), 4 + (2-1), etc.
    return None  # Handle unexpected cases gracefully

In [12]:
code_HOUSEHOLD_TYPE['HOUSEHOLD_TYPE_num'] = code_HOUSEHOLD_TYPE['Code'].apply(household_num)
learn_data['HOUSEHOLD_TYPE'] = learn_data['HOUSEHOLD_TYPE'].apply(household_num)
test_data['HOUSEHOLD_TYPE'] = test_data['HOUSEHOLD_TYPE'].apply(household_num)

In [13]:
def combine_columns(primary_col, fallback_col):
    """Combine two columns, filling missing values in the primary column with values from the fallback column."""
    return primary_col.fillna(fallback_col) if fallback_col is not None else primary_col

def preprocess_employee_data(data, economic_sector_code, work_description_map):
    # Extract numeric values from specific string columns
    data["employee_count"] = data["employee_count"].str.extract(r'tr_(\d)')[0].astype("Int64")
    data["Employer_category"] = data["Employer_category"].str.extract(r'ct_(\d)')[0].astype("Int64")
    
    # Merge with economic sector codes
    data = data.merge(economic_sector_code, left_on="Economic_sector", right_on="Code", how="left")
    
    # Merge with work description map and clean up columns
    data = data.merge(work_description_map, left_on="work_description", right_on="N3", how="left")
    data.drop(["work_description", "N3", "N2"], axis=1, inplace=True)
    data["work_description"] = data["N1"].str.extract(r'csp_(\d)')[0].astype("Int64")
    data.drop("N1", axis=1, inplace=True)
    
    # Combine relevant columns for contracts and pay
    data["emp_contract"] = combine_columns(data["emp_contract"], data["former_emp_contract"])
    data["Pay"] = combine_columns(data["Pay"], data["RETIREMENT_PAY"])
    
    return data

# Apply preprocessing to both learn and test datasets
learn_data = preprocess_employee_data(learn_data, code_Economic_sector, code_work_description_map)
test_data = preprocess_employee_data(test_data, code_Economic_sector, code_work_description_map)

In [14]:
def categorize_retirement_age(data):
    # Ensure the retirement_age column is numeric
    data['retirement_age'] = pd.to_numeric(data['retirement_age'], errors='coerce')
    
    # Define the bins and corresponding labels
    bins = [0, 57, 60, 61, 63, 65, float('inf')]  # Specify edges for the ranges
    labels = ['<57', '57-59', '60', '61-62', '63-64', '65+']  # Labels for ranges

    # Categorize retirement_age into retirement_age_cat
    data['retirement_age_cat'] = pd.cut(
        data['retirement_age'], 
        bins=bins, 
        labels=labels, 
        right=False,  # Left-closed intervals
        include_lowest=True
    )
    
    # Ensure missing values in retirement_age_cat are handled properly
    data['retirement_age_cat'] = data['retirement_age_cat'].astype(object)  # Avoid ambiguity with NA
    
    # Handle exact matches for 60 and 65
    data.loc[data['retirement_age'] == 60, 'retirement_age_cat'] = '60'

# Apply the function to both datasets
categorize_retirement_age(learn_data)
categorize_retirement_age(test_data)   

In [16]:
# Function to preprocess column types
def preprocess_column_types(data):
    data["sex"] = pd.factorize(data["sex"])[0]
    data["studying"] = data["studying"].astype("int64")
    data["Sports_Category"] = data["Sports_Category"].fillna(0).astype("int64")
    
    # List of columns to convert to Int64
    int_columns = ["JOB_REG", "FORMER_REG", "retirement_age", "WORKING_HOURS", "Economic_sector_num", "Pay"]
    
    for col in int_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')
    
    return data

# Apply the function to both datasets
learn_data = preprocess_column_types(learn_data)
test_data = preprocess_column_types(test_data)

In [127]:
def replace_na_with_category(data, column_name):
    # Convert the column to categorical
    data[column_name] = data[column_name].astype('category')
    
    # Define categories to add
    additional_categories = ['Unemployed', 'Retired_Missing', 'Employed_Missing']
    
    # Add the specified categories
    data[column_name] = data[column_name].cat.add_categories(additional_categories)
    
    # Assign categories based on conditions
    data.loc[(data[column_name].isna()) & (data['JOB_42'].astype(str).str.startswith('csp_7')), column_name] = 'Retired_Missing'
    data.loc[(data[column_name].isna()) & (data['act_num'] == 1), column_name] = 'Employed_Missing'
    data.loc[(data[column_name].isna()) & ((data['JOB_42'].astype(str).str.startswith('csp_8')) | (data['act_num'] == 2)), column_name] = 'Unemployed'

# List of columns to process
columns_to_process = [
    "emp_contract", "TYPE_OF_CONTRACT", "WORK_CONDITION", 
    "labor_force_status", "Economic_sector_num", "REG_JOB", 
    "REG_FORMER", "work_description", "retirement_age_cat"
]

# Apply the function to each column for both datasets
for column in columns_to_process:
    replace_na_with_category(learn_data, column)
    replace_na_with_category(test_data, column)

In [129]:
def fill_missing_values(data, column, unemployed_value, default_value):
    data[column] = data.apply(
        lambda row: unemployed_value if pd.isna(row[column]) and 
                        (str(row['JOB_42']).startswith('csp_8') or row['act_num'] == 2)  # unemployed
                    else (default_value if pd.isna(row[column]) else row[column]),
        axis=1
    )

# Fill missing values for both datasets
fill_missing_values(learn_data, 'Employer_category', unemployed_value=10, default_value=0)
fill_missing_values(learn_data, 'employee_count', unemployed_value=7, default_value=0)

fill_missing_values(test_data, 'Employer_category', unemployed_value=10, default_value=0)
fill_missing_values(test_data, 'employee_count', unemployed_value=7, default_value=0)

In [130]:
def clean_data(data, drop_columns):
    # Replace JOB_42 with FORMER_JOB_42 where applicable
    data.loc[data['JOB_42'].astype(str).str.startswith('csp_7'), 'JOB_42'] = data['FORMER_JOB_42']
    
    # Fill missing Pay and WORKING_HOURS for unemployed
    data.loc[(data['emp_contract'] == 'Unemployed') & (data['Pay'].isna()), 'Pay'] = 0
    data.loc[(data['emp_contract'] == 'Unemployed') & (data['WORKING_HOURS'].isna()), 'WORKING_HOURS'] = 0
    
    # Drop unnecessary columns
    return data.drop(columns=drop_columns)

# Define columns to drop for each dataset
drop_columns = [
    "act", "former_emp_contract", "RETIREMENT_PAY", "retirement_age", 
    "FORMER_JOB_42", "Economic_sector", "Code", "Libellé", 
    "Nomenclature", "X", "Y", "INSEE_CODE"
]

# Apply the cleaning function
learn_data = clean_data(learn_data, drop_columns)
test_data = clean_data(test_data, drop_columns)


# Handling Missing Data

In [ ]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)  #% of missing values
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)  #create result table
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

print(missing_values_table(learn_data)) 
print(missing_values_table(test_data)) 

## Replacing missing values with mean

Replacement with values using external data for pay and working hours

In [144]:
# Define job-specific pay and working hours
job_defaults = {
    'csp_1': {'Pay': 50000, 'WORKING_HOURS': 2860},  # Agriculture
    'csp_2_1': {'Pay': 24000, 'WORKING_HOURS': 2288},  # Artisans
    'csp_2_2': {'Pay': 39937, 'WORKING_HOURS': 2444},  # Commerçant
    'csp_2_3': {'Pay': 58248, 'WORKING_HOURS': 2704},  # Chefs d'enterprise
}

# Function to fill missing values for Pay and WORKING_HOURS
def fill_job_defaults(data, job_defaults):
    for job, defaults in job_defaults.items():
        if job == 'csp_1':
            # Special case: Handle 'startswith' condition for 'csp_1'
            job_condition = data['JOB_42'].astype(str).str.startswith(job)
        else:
            # Exact match for other job keys
            job_condition = data['JOB_42'] == job
        
        # Fill missing Pay and WORKING_HOURS
        data.loc[job_condition & data['Pay'].isna(), 'Pay'] = defaults['Pay']
        data.loc[job_condition & data['WORKING_HOURS'].isna(), 'WORKING_HOURS'] = defaults['WORKING_HOURS']

# Apply the function to the datasets
fill_job_defaults(learn_data, job_defaults)
fill_job_defaults(test_data, job_defaults)


In [ ]:
missing_values_learn_data = missing_values_table(learn_data)
missing_values_test_data = missing_values_table(test_data)

# Convert the missing values table to LaTeX format
latex_learn_data = missing_values_learn_data.to_latex(index=False, float_format="%.2f")
latex_test_data = missing_values_test_data.to_latex(index=False, float_format="%.2f")

with open("./missing_values_learn_data.tex", "w") as file:
    file.write(latex_learn_data)
with open("./missing_values_test_data.tex", "w") as file:
    file.write(latex_test_data)

print(missing_values_table(learn_data)) 
print(missing_values_table(test_data)) 

# Sorting for numeric

In [151]:
# List of columns to factorize
columns_to_factorize = [  
    'JOB_42', 'REG_JOB', 'Economic_sector_num', 'work_description', 
    'retirement_age_cat', 'REG_FORMER', 'TYPE_OF_CONTRACT', 
    'WORK_CONDITION', 'labor_force_status', 'emp_contract'
]  

# Function to sort and factorize columns
def sort_and_factorize(data, columns):
    for column in columns:
        data = data.sort_values(by=column, ascending=True)
        data[column] = pd.factorize(data[column])[0]
    return data.sort_values(by='PRIMARY_KEY', ascending=True)

# Apply the function to learn_data and test_data
learn_data = sort_and_factorize(learn_data, columns_to_factorize)
test_data = sort_and_factorize(test_data, columns_to_factorize)

# Building the pipeline

In [154]:
# Custom Transformer for Handling Imputation with Averages
class ImputeMissingValues(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Calculate averages for non-missing, non-zero values
        self.avg_working_hours = (
            X.loc[X["WORKING_HOURS"].notna() & (X["WORKING_HOURS"] != 0)]
            .groupby("JOB_42")["WORKING_HOURS"]
            .mean()
            .round()
        )
        self.avg_pay = (
            X.loc[X["Pay"].notna() & (X["Pay"] != 0)]
            .groupby("JOB_42")["Pay"]
            .mean()
            .round()
        )
        return self

    def transform(self, X, y=None):
        # Make a copy of X to avoid modifying the original dataframe
        X = X.copy()
        
        # Impute WORKING_HOURS using averages
        X["WORKING_HOURS"] = X.apply(
            lambda row: self.avg_working_hours.get(row["JOB_42"], np.nan)
            if pd.isnull(row["WORKING_HOURS"]) else row["WORKING_HOURS"],
            axis=1
        )
        
        # Impute Pay using averages
        X["Pay"] = X.apply(
            lambda row: self.avg_pay.get(row["JOB_42"], np.nan)
            if pd.isnull(row["Pay"]) else row["Pay"],
            axis=1
        )
        
        return X
   

In [156]:
# Custom Transformer for MICE (Multiple Imputation by Chained Equations) Imputation
class MICEImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_impute):
        self.columns_to_impute = columns_to_impute
        self.imputer = IterativeImputer(random_state=100)
        
    def fit(self, X, y=None):
        # Fit the imputer only on the specified columns
        self.imputer.fit(X[self.columns_to_impute])
        return self
    
    def transform(self, X):
        # Transform only the specified columns
        X_imputed = X.copy()
        X_imputed[self.columns_to_impute] = self.imputer.transform(X[self.columns_to_impute])
        return X_imputed

In [158]:
# Specify the columns that need MICE imputation
columns_to_impute = ["Employer_category", "employee_count"]

# Define the combined pipeline
pipeline = Pipeline([
    ('impute_averages', ImputeMissingValues()),  # Step 1: Impute using averages
    ('mice_imputer', MICEImputer(columns_to_impute=columns_to_impute)),  # Step 2: MICE imputation
])


## Linear model

In [161]:
#for linear regression I need only number 

#do one-hot encoding for "labor_force status", but with catagegory being 0, Employed_missing, Unemployed and Other (=only being 6%)
#do one hot-encodung for "type_of_contratct" with CDI, Unemployed, Employed_Missing, Other (=only being 12%)
#do one_hot_encding for "work_condition" with Unemployed, P, Other(=only being 17%)
#do one hot_econding for "emp_contract", with EMP1-6, Unemployed, EMP2-1, Other (=being 12%)
#can keep as is for "highest_credential" because the ordinality make sense here for education
#same for the location of insee code 
#use "work condition" instead of JOB_42 and then do one-hot-encdoding
#do one-hot-encoding for "household_type"
#for "act" do one hot encoding but maybe merge stay at home people with inactif + drop less than 14years olds as nobody in our data is 
#do one hot encoding for "sport" but with a "Other" category
#do one hot encdoding for "REG_JOB" and "FORMER_REG"
#do one hot encdoding for "employer_category" with 1 to 7 being "Others
#can keep it as is for "employee count" because here the ordinality make sense
#do one hot encoding for "Economic_sector_num" and "work_description" 

In [163]:
learn_lin = pipeline.fit_transform(learn_data)

In [ ]:
# check
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)  #% of missing values
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)  #create result table
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

print(missing_values_table(learn_lin)) 

In [167]:
#this gives 0 and 1
def preprocess_for_regression(data):
    # Initialize OneHotEncoder with sparse output disabled to return dense arrays
    ohe = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
    
    # List of columns to encode
    categorical_columns = [
        'HOUSEHOLD_TYPE', 'act_num', 'Sports_Category', 
        'Employer_category', "employee_count",
        'JOB_42', 'REG_JOB', 'Economic_sector_num', 
        'work_description', 'REG_FORMER', 'TYPE_OF_CONTRACT',
        'WORK_CONDITION', 'labor_force_status', 'emp_contract'
    ]

    # List of columns to encode categorical_columns = [
        #'labor_force_status', 'TYPE_OF_CONTRACT', 'WORK_CONDITION', 
        #'emp_contract', 'HOUSEHOLD_TYPE', 'act_num', 'Sports_Category', 
        #'REG_JOB', 'REG_FORMER', 'Employer_category', 'Economic_sector_num', 
        #'work_description' ]
    
    # Apply the encoder
    encoded_data = ohe.fit_transform(data[categorical_columns])
    
    # Create a DataFrame for the encoded features
    encoded_feature_names = ohe.get_feature_names_out(categorical_columns)
    encoded_df = pd.DataFrame(encoded_data, columns=encoded_feature_names, index=data.index)
    
    # Drop original categorical columns from the dataset
    data = data.drop(columns=categorical_columns)
    
    # Merge encoded features with the original dataset
    processed_data = pd.concat([data, encoded_df], axis=1)
    
    return processed_data


# Apply preprocessing to learn and test datasets
learn_lin = preprocess_for_regression(learn_lin)
#test_data = preprocess_for_regression(test_data)


In [169]:
x_train_lin = learn_lin.drop(columns=["target"])
y_train_lin = learn_lin["target"]

In [171]:
X_train_lin, X_test_lin, Y_train_lin, Y_test_lin = train_test_split(x_train_lin,
                                                    y_train_lin, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [ ]:
# Example GridSearchCV for Linear Regression
param_grid = {'fit_intercept': [True, False]}  # Define your hyperparameter grid

# Create a Linear Regression model
lr = LinearRegression()

# Perform grid search with cross-validation
lin_res = GridSearchCV(estimator=lr, param_grid=param_grid, cv=5)

# Fit the GridSearchCV model
lin_res.fit(X_train_lin, Y_train_lin)

# Output the best model and its score
print(f"Best model: {lin_res.best_estimator_}")
print(f"Best score: {lin_res.best_score_}")


In [ ]:
train_predictions = lin_res.predict(X_train_lin)
print("R² for training set: ", r2_score(Y_train_lin, train_predictions))
print("RMSE on the learning set:", root_mean_squared_error(Y_train_lin, lin_res.predict(X_train_lin)))

In [ ]:
train_predictions = lin_res.predict(X_train_lin)
test_predictions = lin_res.predict(X_test_lin)

plt.figure(figsize=(8, 6))
sns.scatterplot(x=Y_test_lin, y=test_predictions, alpha=0.6, color="blue")
plt.plot([y_train_lin.min(), y_train_lin.max()], [y_train_lin.min(), y_train_lin.max()], "--r", linewidth=2)  # Ideal predictions
plt.xlabel('True Target Value')
plt.ylabel('Predicted Target Value')
plt.title('Prediction vs True Values on Test Set')
plt.savefig("./Prediction_vs_True_linear.png", format='png', dpi=300)  
plt.show()

## Random Forest model

In [179]:
learn_rf = pipeline.fit_transform(learn_data)

In [180]:
x_train_rf = learn_rf.drop(columns=["target"])
y_train_rf = learn_rf["target"]

X_train_rf, X_test_rf, Y_train_rf, Y_test_rf = train_test_split(x_train_rf,
                                                    y_train_rf, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [183]:
param_grid_rf = {     # Number of trees in the forest
    'max_depth': [5, 10, 30, 50],    
    'min_samples_split': [5, 10, 20]  
}

# Define cross-validation with 5 folds
cv_folds = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the RandomForestRegressor model
rf = RandomForestRegressor(random_state=42)  #need to add that in the pipeline 

# Set up the GridSearchCV for RandomForestRegressor with appropriate scoring metric
rf_search = GridSearchCV(rf, param_grid_rf, cv=cv_folds, n_jobs=-1, scoring="neg_mean_squared_error")

In [185]:
# Fit the model on the training data
rf_res = rf_search.fit(X_train_rf, Y_train_rf)

In [ ]:
# Get the best model after GridSearchCV
best_model_rf = rf_res.best_estimator_

# Output the best model and its score
print(f"Best model: {best_model_rf}")
print(f"Best score: {rf_res.best_score_}")

In [ ]:
cv_scores = cross_val_score(best_model_rf, X_train_rf, Y_train_rf, cv=5, scoring='r2')
print(f"Cross-validated R2 scores: {cv_scores}")
print(f"Average R2: {np.mean(cv_scores):.2f}")

In [ ]:
train_predictions = best_model_rf.predict(X_train_rf)
test_predictions = best_model_rf.predict(X_test_rf)

print("R² for training set: ", r2_score(Y_train_rf, train_predictions))
print("R² for test set: ", r2_score(Y_test_rf, test_predictions))
print("RMSE on the learning set:", root_mean_squared_error(Y_train_rf, best_model_rf.predict(X_train_rf)))
print("RMSE on the test set:", root_mean_squared_error(Y_test_rf, best_model_rf.predict(X_test_rf)))

In [193]:
error_metrics = pd.DataFrame({
    'Metric': ['R²', 'RMSE'],
    'Train': [r2_score(Y_train_rf, train_predictions), root_mean_squared_error(Y_train_rf, best_model_rf.predict(X_train_rf))],
    'Test': [r2_score(Y_test_rf, test_predictions), root_mean_squared_error(Y_test_rf, best_model_rf.predict(X_test_rf))]
})

# Set 'Metric' as the index to have a clean format
error_metrics.set_index('Metric', inplace=True)

# Convert to LaTeX format
error_metrics_table = error_metrics.to_latex(index=True, float_format="%.2f")

with open("./error_metrics.tex", "w") as file:
    file.write(error_metrics_table)

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=Y_test_rf, y=test_predictions, alpha=0.6, color="blue")
plt.plot([y_train_rf.min(), y_train_rf.max()], [y_train_rf.min(), y_train_rf.max()], "--r", linewidth=2)  # Ideal predictions
plt.xlabel('True Target Value')
plt.ylabel('Predicted Target Value')
plt.title('Prediction vs True Values on Test Set')
plt.savefig("./Prediction_vs_True_rf.png", format='png', dpi=300)  
plt.show()

In [ ]:
# Calculate correlation
correlation_matrix = learn_data.corr()

# Find correlations with the target variable
target_corr = correlation_matrix['target'].sort_values(ascending=False)

print(target_corr)

target_corr_table = target_corr.to_latex(index=False, float_format="%.2f")

with open("./target_corr.tex", "w") as file:
    file.write(target_corr_table)

In [ ]:
# Get feature importances
feature_importances = pd.DataFrame({
    'Feature': x_train_rf.columns,
    'Importance': best_model_rf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(feature_importances)

feature_importances_table = feature_importances.to_latex(index=False, float_format="%.2f")

with open("./feature_importances.tex", "w") as file:
    file.write(feature_importances_table)

# Experiments

## these didn't work yet - ignore

In [ ]:
results = rf_res.cv_results_
params = results['params']

# Extract individual fold scores
fold_scores = []
for i in range(10):
    fold_scores.append(-results[f'split{i}_test_score'])  # Negate for RMSE


# Create a DataFrame to store the results, including fold information
df = pd.DataFrame(params)
for i, fold_score in enumerate(fold_scores):
    df[f'fold_{i}'] = fold_score

# Melt the DataFrame to long format for easier plotting
df_melted = df.melt(id_vars=['max_depth', 'min_samples_split'], var_name='fold', value_name='rmse')

# Create a line plot for RMSE as a function of min_samples_split, with different colors for each max_depth
sns.lineplot(x='min_samples_split', y='rmse', hue='max_depth', data=df_melted, errorbar='se')


In [ ]:
cv_folds = StratifiedKFold(10, shuffle=True, random_state=0)
dt = DecisionTreeRegressor(random_state=0)
dt_grid = {'min_samples_split': [2, 5] + list(range(10, 250,5)),
           'max_depth': [2, 5, 10, 15, 20]}

rmse_train = []
rmse_test = []
rmse_cv = []
params_cv = []
for k in range(50):
    ## we use k for the random state to introduce different CV splits
    cv_folds = StratifiedKFold(10, shuffle=True, random_state=k)
    dt = RandomForestRegressor(random_state=0)
    dt_cv = GridSearchCV(dt, dt_grid, cv=cv_folds, n_jobs=-1, 
                     scoring='neg_root_mean_squared_error')
    dt_cv.fit(X_train, Y_train)
    rmse_cv.append(-dt_cv.best_score_)
    rmse_train.append(root_mean_squared_error(Y_train, dt_cv.predict(X_train)))
    rmse_test.append(root_mean_squared_error(Y_test, dt_cv.predict(X_test)))
    params_cv.append(dt_cv.best_params_)
    
#%% 
all_results = pd.DataFrame({'run': list(range(len(rmse_cv))),
                            'CV': rmse_cv,
                            'train': rmse_train,
                            'test': rmse_test
    })

res = all_results.melt('run',var_name='Estimate',value_name='Accuracy')
sns.displot(data=res,x='Accuracy',kind='kde',hue='Estimate',fill=True)

param_results = pd.DataFrame({'run': list(range(len(rmse_cv))),
                              'max_depth': [x['max_depth'] for x in params_cv],
                              'min_samples_split': [x['min_samples_split'] for x in params_cv]
                              })

sns.displot(data=param_results,x='max_depth')

sns.displot(data=param_results,x='min_samples_split')


## DO NOT RUN - just look at output lol

In [ ]:
# Sample dataset (replace this with your own dataset)
# X = features, y = target variable
# X, y = your_data.drop("target", axis=1), your_data["target"]

# Split dataset into training and testing sets
nX_train, nX_test, ny_train, ny_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

pipeline.fit_transform(nX_train, ny_train)

# Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# 1. Fit the model
rf.fit(nX_train, ny_train)

# 2. Predictions
ny_pred = rf.predict(nX_test)

# 3. Performance Metrics
mse = mean_squared_error(ny_test, ny_pred)
r2 = r2_score(ny_test, ny_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

# Graphical Visualization of Predictions
plt.figure(figsize=(10, 6))
plt.scatter(ny_test, ny_pred, alpha=0.7, color="blue")
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], "--r", linewidth=2)  # Ideal predictions
plt.title("Actual vs Predicted Values")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.grid()
plt.show()

# 4. Resampling using cross-validation
cv_scores = cross_val_score(rf, x_train, y_train, cv=5, scoring='r2')
print(f"Cross-validated R2 scores: {cv_scores}")
print(f"Average R2: {np.mean(cv_scores):.2f}")

# 5. Hyperparameter Tuning with Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, scoring='r2', verbose=2)
grid_search.fit(nX_train, ny_train)

print("Best Parameters:", grid_search.best_params_)

# Evaluate the best model
best_rf = grid_search.best_estimator_
print(f"Best model: {best_rf}")


In [ ]:
#Best Parameters: {'bootstrap': True, 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}

pipeline.transform(nX_test)

y_pred_best = best_rf.predict(nX_test)

# Graphical Visualization of Best Model Predictions
plt.figure(figsize=(10, 6))
sns.histplot(ny_test - y_pred_best, kde=True, bins=30, color="green")
plt.title("Residual Distribution of Best Model")
plt.xlabel("Residuals (y_test - y_pred_best)")
plt.ylabel("Frequency")
plt.grid()
plt.show()


# Evaluation

In [142]:
test_rf = pipeline.transform(test_data)
test_predictions = best_model_rf.predict(test_rf)  # Replace with your model's prediction method

predictions_df = pd.DataFrame({
    'PRIMARY_KEY': test_rf['PRIMARY_KEY'], 
    'target': test_predictions
})

# Save the predictions to a CSV file
predictions_df.to_csv('predictions.csv', index=False, sep=',', float_format='%.6f')  # Save with US decimal notation

In [ ]:
predictions = pd.read_csv("predictions.csv")
predictions
